# Setup

In [19]:
import wandb
import tensorflow as tf
from sklearn.datasets import load_breast_cancer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os

# Data

In [20]:
data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33)
N, D = X_train.shape
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Model

In [21]:
# Configure the wandb sweep
sweep_config = {
    'method': 'bayes',
    'metric': {
      'name': 'val_loss',
      'goal': 'minimize'   
    },
    'parameters': {
        'learning_rate': {
            'values': [1, 0.1, 0.01, 0.001, 0.0001]
        },
        'epochs': {
            'values': [10, 20, 50, 100, 200, 500, 1000]
        },
        'batch_size': {
            'values': [16, 32, 64, 128, 256]
        },
        'optimizer': {
            'values': ['adam', 'nadam', 'sgd', 'rmsprop']
        },
    }
}

In [22]:
# Initialise sweep
sweep_id = wandb.sweep(sweep_config, project='tensorflow-test', entity='kavp')



Create sweep with ID: dmpwxa3f
Sweep URL: https://wandb.ai/kavp/tensorflow-test/sweeps/dmpwxa3f


In [23]:
# Mega function to define and train model and log results (used by the sweep)
def sweep_func():
    # Default hyperparameter values
    config_defaults = {
        'learning_rate': 0.001,
        'epochs': 100,
        'batch_size': 128,
        'optimizer': 'adam',
        'eager_mode': False,
    }

    # Initialise run
    wandb.init(config=config_defaults)

    # Variable to hold the sweep values
    config = wandb.config
    
    if config['eager_mode'] == True:
        tf.compat.v1.enable_eager_execution()
    elif config['eager_mode'] == False:
        tf.compat.v1.disable_eager_execution()
    else:
        raise ValueError('eager_mode property of wandb config could not be determined.') 

    # Define our own sequential model
    model = tf.keras.models.Sequential([
        # Layers
        tf.keras.layers.Input(shape=(D,)),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])

    model.compile(
        optimizer=config['optimizer'],
        loss='binary_crossentropy',
        metrics=['accuracy'],
        run_eagerly=config['eager_mode'],
    )

    with tf.compat.v1.Session() as sess:
        r = model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=config['epochs'], batch_size=config['batch_size'])
        wandb.tensorflow.log(tf.compat.v1.summary.merge_all())
        wandb.log({'loss': r.history['loss'][-1], 'val_loss': r.history['val_loss'][-1], 'accuracy': r.history['accuracy'][-1], 'val_accuracy': r.history['val_accuracy'][-1]})

        wandb_data = [[x,y] for (x,y) in zip(np.arange(0, config['epochs'], 1), r.history['loss'])]
        table = wandb.Table(data=wandb_data, columns = ["epoch", "loss"])
        wandb.log({"loss_against_epochs" : wandb.plot.line(table, "epoch", "loss", title="Training loss")})

        wandb_data = [[x,y] for (x,y) in zip(np.arange(0, config['epochs'], 1), r.history['val_loss'])]
        table = wandb.Table(data=wandb_data, columns = ["epoch", "val_loss"])
        wandb.log({"val_loss_against_epochs" : wandb.plot.line(table, "epoch", "val_loss", title="Validation loss")})

        wandb_data = [[x,y] for (x,y) in zip(np.arange(0, config['epochs'], 1), r.history['accuracy'])]
        table = wandb.Table(data=wandb_data, columns = ["epoch", "accuracy"])
        wandb.log({"accuracy_against_epochs" : wandb.plot.line(table, "epoch", "accuracy", title="Training accuracy")})

        wandb_data = [[x,y] for (x,y) in zip(np.arange(0, config['epochs'], 1), r.history['val_accuracy'])]
        table = wandb.Table(data=wandb_data, columns = ["epoch", "val_accuracy"])
        wandb.log({"val_accuracy_against_epochs" : wandb.plot.line(table, "epoch", "val_accuracy", title="Validation accuracy")})

        # Save model
        model.save(os.path.join(wandb.run.dir, 'model.h5'))

In [ ]:
wandb.agent(sweep_id, sweep_func)

In [25]:
wandb.finish()

# Predictions

In [36]:
# Load a good run
good_run_path = 'kavp/tensorflow-test/1kc8ars5/'
run = wandb.Api().run(good_run_path)
# Load model
best_model = wandb.restore('model.h5', run_path=good_run_path)
# Load config
config = run.config

# Define a new model to load weights into
model = tf.keras.models.Sequential([
    # Layers
    tf.keras.layers.Input(shape=(D,)),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])

model.compile(
        optimizer=config['optimizer'],
        loss='binary_crossentropy',
        metrics=['accuracy'],
        run_eagerly=config['eager_mode'],
    )


# Load its weights into the new model
model.load_weights(best_model.name)

In [37]:
# Make predictions with model
P = model.predict(X_test).round().flatten()
print(P)

6/6 [==============================] - 0s 400us/step
[0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0.
 0. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1.
 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0.
 1. 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 0. 1.
 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 0. 0. 0. 0.
 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1.]


In [38]:
print('Manually calculated accuracy:', np.mean(P == y_test))
print('Model evaluated output:', model.evaluate(X_test, y_test))

Manually calculated accuracy: 0.9574468085106383
6/6 [==============================] - 0s 1ms/step - loss: 0.0593 - accuracy: 0.9574
Model evaluated output: [0.059262923896312714, 0.957446813583374]
